# References

Medical neural networks:
https://glassboxmedicine.com/

Comparison of ResNet50 and VGG19 and training from stratch for X-ray images dataset:
https://www.sciencedirect.com/science/article/pii/S2666285X21000558

Tensorboard confusion matrix:
https://towardsdatascience.com/exploring-confusion-matrix-evolution-on-tensorboard-e66b39f4ac12

Pre-processing and modeling pipelines (ResNet50):
https://towardsdatascience.com/time-to-choose-tensorflow-data-over-imagedatagenerator-215e594f2435

Image data input pipelines:
https://towardsdatascience.com/what-is-the-best-input-pipeline-to-train-image-classification-models-with-tf-keras-eb3fe26d3cc5

Split TF datasets:
https://towardsdatascience.com/how-to-split-a-tensorflow-dataset-into-train-validation-and-test-sets-526c8dd29438

Transfer learning with EfficientNet:
https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

Training greyscale images using transfer learning:
https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images

Multi-label vs multi-class classification:
https://glassboxmedicine.com/2019/05/26/classification-sigmoid-vs-softmax/

Element-wise sigmoid:
https://www.programcreek.com/python/example/93769/keras.backend.sigmoid

Element-wise sigmoid:
https://stackoverflow.com/questions/52090857/how-to-apply-sigmoid-function-for-each-outputs-in-keras

# Setup

### Install packages

In [73]:
# Install additional packages

!pip install tf_keras_vis

In [74]:
import os
import pathlib
import re
from glob import glob
import random
import time
import zipfile
import imageio
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import time
from itertools import islice, count

from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.keras.backend import clear_session
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB2
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, RandomTranslation
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam, SGD, Nadam, Adagrad, RMSprop
from tensorflow.keras.regularizers import l1_l2, l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.initializers import he_normal, glorot_normal
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K # element-wise sigmoid
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.saliency import Saliency

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

### Check GPU and CPU instances

In [75]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment
# that evaluates operations immediately, without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print(f"tensorflow version {tf.__version__}")
print(f"keras version {tf.keras.__version__}")
print(f"Eager Execution Enabled: {tf.executing_eagerly()}\n")

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

tensorflow version 2.8.0
keras version 2.8.0
Eager Execution Enabled: True

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of replicas: 1
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
All Physical Devices [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [76]:
# Check which GPU is assigned to the session
!nvidia-smi

Fri Apr 29 23:06:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W /  70W |   8982MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Set seeds and mount Google Drive

In [77]:
# Ensure replicable results
import os
import random as rn
SEED = 109
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

In [78]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to Google Drive
base_path = '/content/drive/My Drive/cs109b_final_project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocessing

### Load image data into TF datasets

In [79]:
# Get image filenames

# Path to images
images_path = base_path + 'images_rescaled_subsample'

# Filenames with full absolute paths
images_filenames_full_path = glob(images_path + '/*')

# Sort to be consistent
images_filenames_full_path.sort()

# Get terminal filenames
images_filenames = [re.sub('^(.*[/])', '', x) for x in images_filenames_full_path]

In [80]:
# Function to load the image data into TF datasets

def make_tf_image_dataset(filenames, image_size):

  def parse_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image

  filenames_ds = tf.data.Dataset.from_tensor_slices(filenames)
  images_ds = filenames_ds.map(parse_image, num_parallel_calls=AUTOTUNE)

  return images_ds

In [81]:
# Load the image and label data into TF datasets
# NOTE: this will load the grayscale image repeated 3 times (i.e., 3 channels)

IMG_SIZE = 256

images_ds = make_tf_image_dataset(filenames=images_filenames_full_path, image_size=IMG_SIZE)

print("images_ds shape: ", images_ds)
print("number of images parsed: ", images_ds.cardinality().numpy())

images_ds shape:  <ParallelMapDataset element_spec=TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None)>
number of images parsed:  1100


### Load label data into TF datasets

#### From "train_relabeled.csv" and "valid_relabeled.csv"

In [13]:
# # Load label data from "train_relabeled.csv" and "valid_relabeled.csv"

# # Limit scope to 8 diseases in original paper
# columns_to_keep = ['Path', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']

# # Load "train_relabeled.csv" and "valid_relabeled.csv"
# train_labels_df = pd.read_csv(google_drive_path + 'label_data/train_relabeled.csv')[columns_to_keep]
# valid_labels_df = pd.read_csv(google_drive_path + 'label_data/valid_relabeled.csv')[columns_to_keep]

# # Examine first 5 rows of training dataframe
# print(f'Shape of train_labels_df: {train_labels_df.shape}')
# display(train_labels_df.head())

# # Examine first 5 rows of validation dataframe
# print(f'Shape of valid_labels_df: {valid_labels_df.shape}')
# display(valid_labels_df.head())

# # Stack the two DataFrames
# labels_df = pd.concat([train_labels_df, valid_labels_df], ignore_index=True, axis=0)

# # Remove the 'images/' prefix from the Path column
# labels_df['Path'] = labels_df['Path'].apply(lambda x: re.sub('^(.*[/])', '', x))

# # Examine first 5 rows of combined dataframe
# print(f'Shape of labels_df: {labels_df.shape}')
# display(labels_df.head())

#### From "Data_Entry_2017_v2020.csv"

In [82]:
# Load Data_Entry_2017_v2020.csv

labels_path = base_path + 'meta_data/Data_Entry_2017_v2020.csv'
data_entry_df = pd.read_csv(labels_path)

# Examine first 5 rows of dataframe
print(f'Shape of data_entry_df: {data_entry_df.shape}')
display(data_entry_df.head())

Shape of data_entry_df: (112120, 11)


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168


In [83]:
# Determine the number of unique label combinations
disease_combinations = data_entry_df['Finding Labels'].unique()
print(f'Number of unique disease combinations: {len(disease_combinations)}\n')

# Split up disease_combinations into individual diseases - get number of unique labels
disease_combinations_split = [combination.split('|') for combination in disease_combinations]
diseases = list(set([disease for observation in disease_combinations_split for disease in observation]))

# Sanity check - how many unique diseases are present in the data
print(f'Number of unique diseases (including No Finding): {len(diseases)}\n')
print('Unique diseases (including No Finding): \n')
for disease in diseases:
  print(disease)

Number of unique disease combinations: 836

Number of unique diseases (including No Finding): 15

Unique diseases (including No Finding): 

Pneumothorax
Emphysema
Mass
Effusion
Consolidation
Nodule
Infiltration
Pneumonia
Hernia
Pleural_Thickening
No Finding
Fibrosis
Edema
Atelectasis
Cardiomegaly


In [84]:
# One hot encode label data

# Rename image column
data_entry_df = data_entry_df.rename({'Image Index': 'image_filename'}, axis=1)

# Limit scope to 8 diseases in original paper plus 'No Finding'
diseases_to_keep = ['No Finding', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']
diseases = [disease for disease in diseases if disease in diseases_to_keep]

# Helper function to create one-hot encoded dataframe
@np.vectorize
def one_hot_disease(label, disease='No Finding'):
  if disease in label:
    return 1
  return 0

# Create one-hot encoded dataframe
one_hot_disease_df = data_entry_df[['image_filename']]
for disease in diseases:
  one_hot_disease_df[disease] = one_hot_disease(data_entry_df['Finding Labels'], disease=disease)

# Rename no finding column
one_hot_disease_df = one_hot_disease_df.rename({'No Finding': 'No_Finding'}, axis=1)

# Save results
one_hot_disease_df.to_csv(base_path + 'output/one_hot_data_entry.csv', index=False)

# Examine first 5 rows of one_hot_disease_df dataframe
print(f'Shape of one_hot_disease_df: {one_hot_disease_df.shape}')
display(one_hot_disease_df.head())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Shape of one_hot_disease_df: (112120, 10)


,image_filename,Pneumothorax,Mass,Effusion,Nodule,Infiltration,Pneumonia,No_Finding,Atelectasis,Cardiomegaly
0,00000001_000.png,0,0,0,0,0,0,0,0,1
1,00000001_001.png,0,0,0,0,0,0,0,0,1
2,00000001_002.png,0,0,1,0,0,0,0,0,1
3,00000002_000.png,0,0,0,0,0,0,1,0,0
4,00000003_001.png,0,0,0,0,0,0,0,0,0


In [85]:
# Subset labels_df to include only subsample

# Logically index labels_df based on subsample filenames
labels_subsample_df = one_hot_disease_df[one_hot_disease_df['image_filename'].isin(images_filenames)]

# Reorder labels to match image order
labels_subsample_df = labels_subsample_df.sort_values('image_filename') 

# Examine first 5 rows of labels_subsample_df dataframe
print(f'Shape of labels_subsample_df: {labels_subsample_df.shape}')
display(labels_subsample_df.head())

Shape of labels_subsample_df: (1100, 10)


,image_filename,Pneumothorax,Mass,Effusion,Nodule,Infiltration,Pneumonia,No_Finding,Atelectasis,Cardiomegaly
151,00000032_037.png,0,0,0,0,1,0,0,0,1
310,00000072_000.png,0,0,0,0,0,0,0,1,0
596,00000147_001.png,0,0,0,0,0,0,0,1,0
608,00000149_006.png,0,0,0,0,0,0,0,1,0
614,00000150_002.png,0,0,0,0,1,1,0,1,0


In [86]:
# Convert labels to TF dataset

# Create label ds
labels_ds = tf.data.Dataset.from_tensor_slices(labels_subsample_df.drop(columns='image_filename'))   

print("labels_ds shape: ", labels_ds)
print("number of labels parsed: ", labels_ds.cardinality().numpy())   

labels_ds shape:  <TensorSliceDataset element_spec=TensorSpec(shape=(9,), dtype=tf.int64, name=None)>
number of labels parsed:  1100


### Combine images and labels

In [87]:
# Zip together the image and label data into one TF dataset

complete_ds = tf.data.Dataset.zip((images_ds, labels_ds))
complete_ds

<ZipDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(9,), dtype=tf.int64, name=None))>

In [88]:
def transform(input_tensor, input_target):    

    print(input_tensor.shape)
    print(input_target.shape)
    # Transform the target
    target_0  = input_target[0]
    target_1  = input_target[1]
    target_2  = input_target[2]
    target_3  = input_target[3]
    target_4  = input_target[4]
    target_5  = input_target[5]
    target_6  = input_target[6]
    target_7  = input_target[7]
    target_8  = input_target[8]

    return (input_tensor), (target_0, target_1, target_2, target_3, target_4, target_5, target_6, target_7, target_8)

In [89]:
complete_alt_ds = complete_ds.map(transform, num_parallel_calls=AUTOTUNE)
complete_alt_ds = complete_alt_ds.prefetch(1)
complete_alt_ds.element_spec[1]

(256, 256, 3)
(9,)


(TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

### Partition TF datasets into training and validation sets

In [90]:
# Function to partition the TF datasets into Training and Validation sets

def partition_tf_data(ds, ds_size, train_split=0.8, val_split=0.2, 
                      test_split=0, shuffle=True, shuffle_size=10000):
  
    # assert(train_split + test_split + val_split) == 1
    assert(train_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=109)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    # test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds #, test_ds

In [91]:
# Partition the TF datasets

DS_SIZE = images_ds.cardinality().numpy()

train_ds, val_ds = partition_tf_data(ds=complete_ds, ds_size=DS_SIZE)

print("Train shape: ", train_ds)
print("Validation shape: ", val_ds)
print("number of training images/labels: ", train_ds.cardinality().numpy())
print("number of validation images/labels: ", val_ds.cardinality().numpy())

Train shape:  <TakeDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(9,), dtype=tf.int64, name=None))>
Validation shape:  <TakeDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(9,), dtype=tf.int64, name=None))>
number of training images/labels:  880
number of validation images/labels:  220


In [92]:
# Partition the TF datasets

DS_SIZE = images_ds.cardinality().numpy()

train_alt_ds, val_alt_ds = partition_tf_data(ds=complete_alt_ds, ds_size=DS_SIZE)

print("Train shape: ", train_alt_ds)
print("Validation shape: ", val_alt_ds)
print("number of training images/labels: ", train_alt_ds.cardinality().numpy())
print("number of validation images/labels: ", val_alt_ds.cardinality().numpy())

Train shape:  <TakeDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None)))>
Validation shape:  <TakeDataset element_spec=(TensorSpec(shape=(256, 256, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), T

### Batching

In [93]:
# Set up batches

BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 32

train_ds_batches = train_ds.shuffle(buffer_size=40000).batch(BATCH_SIZE_TRAIN).prefetch(buffer_size=AUTOTUNE)
val_ds_batches = val_ds.batch(BATCH_SIZE_VAL).prefetch(buffer_size=AUTOTUNE)

print("Train shape: ", train_ds_batches)
print("Validation shape: ", val_ds_batches)

Train shape:  <PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 9), dtype=tf.int64, name=None))>
Validation shape:  <PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 9), dtype=tf.int64, name=None))>


In [94]:
# Set up batches

BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 32

train_alt_ds_batches = train_alt_ds.shuffle(buffer_size=40000).batch(BATCH_SIZE_TRAIN).prefetch(buffer_size=AUTOTUNE)
val_alt_ds_batches = val_alt_ds.batch(BATCH_SIZE_VAL).prefetch(buffer_size=AUTOTUNE)

print("Train shape: ", train_alt_ds_batches)
print("Validation shape: ", val_alt_ds_batches)

Train shape:  <PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None)))>
Validation shape:  <PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpe

### Preprocessing layers

In [95]:
# Pre-processing layers

# Rescale pixels
rescale_layer = Sequential([layers.experimental.preprocessing.Rescaling(1./255)])

# Data augmentation
data_augmentation_layer = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2), 
  RandomZoom(height_factor=(0.2, 0.3), width_factor=(0.2, 0.3)),
  RandomTranslation(0.3, 0.3, fill_mode='reflect', interpolation='bilinear',)
])

# Model Training


### Callbacks

In [96]:
# Callbacks

# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    mode='min', 
    min_delta=0.1, 
    patience=10, 
    restore_best_weights=True, 
    verbose=True)

# Reduce learning rate by factor of 0.5 if val_coef_determination does not improve within 3 epochs
reduce_lrt_plateau = ReduceLROnPlateau(
    monitor='val_loss', 
    mode='min',
    min_delta=0.2,
    factor=0.5,
    patience=3, 
    min_lr=1e-6,
    verbose=2)

# This function keeps the initial learning rate for the first 7 epochs and decreases it exponentially after that
def scheduler(epoch, lr):
    if epoch <= 7:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lrt_scheduler = LearningRateScheduler(scheduler)

### Model architecture

Which model to use for transfer learning?

- ResNet (50, 18)
- VGG (16, 19)
- EfficientNet (B0, B1, B2, B3, B4, B5, B6, B7)

In [97]:
# Define alternative model architecture for element-wise sigmoid

IMG_SIZE = 256

def create_alternative_model(transfer_model='ResNet50'):

  # Transfer models
  if transfer_model=='ResNet50':
      transfer_layer = ResNet50(include_top=False, weights='imagenet', 
                                input_shape=(IMG_SIZE, IMG_SIZE, 3))
  elif transfer_model=='EfficientNetB2':
      transfer_layer = EfficientNetB2(include_top=False, weights="imagenet",
                                      input_shape=(IMG_SIZE, IMG_SIZE, 3))
  elif transfer_model=='VGG16':
      transfer_layer = VGG16(include_top=False, weights="imagenet",
                             input_shape=(IMG_SIZE, IMG_SIZE, 3))
  elif transfer_model=='VGG19':
      transfer_layer = VGG19(include_top=False, weights="imagenet",
                             input_shape=(IMG_SIZE, IMG_SIZE, 3))
      
  # Inputs, Preprocessing, and Transfer layers
  inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  rescale = rescale_layer(inputs)
  augmented = data_augmentation_layer(rescale)
  transfer = transfer_layer(augmented)
  # TRY STUFF HERE

  # Pool for dense output
  pooling = GlobalAveragePooling2D()(transfer)
  dropout = Dropout(0.4)(pooling)
  # TRY STUFF HERE
  
  # Output layer
  out_1 = Dense(units=1, activation='sigmoid')(dropout)
  out_2 = Dense(units=1, activation='sigmoid')(dropout)
  out_3 = Dense(units=1, activation='sigmoid')(dropout)
  out_4 = Dense(units=1, activation='sigmoid')(dropout)
  out_5 = Dense(units=1, activation='sigmoid')(dropout)
  out_6 = Dense(units=1, activation='sigmoid')(dropout)
  out_7 = Dense(units=1, activation='sigmoid')(dropout)
  out_8 = Dense(units=1, activation='sigmoid')(dropout)
  out_9 = Dense(units=1, activation='sigmoid')(dropout)

  model = Model(inputs=inputs, outputs=[out_1, out_2, out_3, out_4, out_5, out_6, out_7, out_8, out_9])

  return model

clear_session()
base_alt_model = create_alternative_model(transfer_model='VGG19')
base_alt_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 256, 256, 3)  0           ['input_2[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 256, 256, 3)  0           ['sequential[0][0]']             
                                                                                                  
 vgg19 (Functional)             (None, 8, 8, 512)    20024384    ['sequential_1[0][0]']       

### Compile

In [98]:
# Compile

# Optimizers
opt = Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# Compile model
base_alt_model.compile(
    optimizer=opt, 
    loss='binary_crossentropy', 
    metrics=['AUC'])

### Train

In [99]:
# Train the model

# Parameters
EPOCHS = 50 

# Training
alt_history = base_alt_model.fit(
    x=train_alt_ds_batches,  
    validation_data=val_alt_ds_batches,
    epochs=EPOCHS,  
    verbose=1,
    use_multiprocessing=True, 
    callbacks=[early_stop, reduce_lrt_plateau, lrt_scheduler]
    )

%time

Epoch 1/50
28/28 [==============================] - 34s 824ms/step - loss: 62.3883 - dense_loss: 9.4224 - dense_1_loss: 8.6915 - dense_2_loss: 9.3228 - dense_3_loss: 8.4721 - dense_4_loss: 6.4210 - dense_5_loss: 2.6701 - dense_6_loss: 1.8542 - dense_7_loss: 10.7346 - dense_8_loss: 4.7998 - dense_auc: 0.4708 - dense_1_auc_1: 0.5456 - dense_2_auc_2: 0.4789 - dense_3_auc_3: 0.4810 - dense_4_auc_4: 0.4733 - dense_5_auc_5: 0.5043 - dense_6_auc_6: 0.5046 - dense_7_auc_7: 0.4689 - dense_8_auc_8: 0.4968 - val_loss: 4.3464 - val_dense_loss: 0.4535 - val_dense_1_loss: 0.3453 - val_dense_2_loss: 0.5938 - val_dense_3_loss: 0.4278 - val_dense_4_loss: 0.5581 - val_dense_5_loss: 0.3672 - val_dense_6_loss: 0.5067 - val_dense_7_loss: 0.5884 - val_dense_8_loss: 0.5057 - val_dense_auc: 0.5000 - val_dense_1_auc_1: 0.5000 - val_dense_2_auc_2: 0.4968 - val_dense_3_auc_3: 0.5000 - val_dense_4_auc_4: 0.5000 - val_dense_5_auc_5: 0.5000 - val_dense_6_auc_6: 0.4876 - val_dense_7_auc_7: 0.5000 - val_dense_8_auc_8

### Save / Load model

In [100]:
# Save model
base_alt_model.save(base_path + 'output/base_alt_model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/cs109b_final_project/output/base_alt_model/assets


# Model Evaluation

### Plot history

In [49]:
from tensorflow.python.ops.gen_logging_ops import histogram_summary
def plot_history(model, title:str=''):
    """Create plots for the training history"""
    
    h = model.history.history
    y1 = h['auc']
    y2 = h['val_auc']
    y3 = h['loss']
    y4 = h['val_loss']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,7))

    x = model.history.epoch
    ax1.plot(x, y1, label='train', linewidth=2)
    ax1.plot(x, y2, label='validation', linewidth=2)
    ax1.set_title('AUC-ROC', fontsize=20)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('AUC-ROC')
    best_auc = np.nanmax(y2)
    ax1.axvline(np.nanargmax(y2),
                ls='--', label=f'best val auc = {best_auc:.2f}')
    ax1.grid(True)
    ax1.legend()

    ax2.plot(x, y3, label='train', linewidth=2)
    ax2.plot(x, y4, label='validation', linewidth=2)
    ax2.set_title('Loss', fontsize=20)
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    best_loss = np.nanmin(y4)
    ax2.axvline(np.nanargmin(y4),
                ls='--', label=f'best val loss = {best_loss:.2f}')
    ax2.grid(True)
    ax2.legend()

    fig.suptitle(title, fontsize=12)

### Performance metrics

How to set up a confusion matrix from a prefetched TF dataset?

In [101]:
# Predict on the test set
pred_alt = base_alt_model.predict(val_alt_ds_batches)
pred_alt_flat = list()
for i in range(len(pred_alt)):
  x = [num for sublist in pred_alt[i] for num in sublist]
  pred_alt_flat.append(x)

print(pred_alt_flat[0])

pred_df_alt = pd.DataFrame(pred_alt_flat).T
pred_df_alt.to_csv('/content/drive/My Drive/pred_alt.csv')
print(pred_df_alt.head(100))

[0.1794232, 0.17958516, 0.17951596, 0.17952952, 0.17943922, 0.1795317, 0.17975439, 0.17958121, 0.17945094, 0.17936449, 0.17916618, 0.17975508, 0.17944434, 0.17934006, 0.17982647, 0.17942837, 0.17986959, 0.1793774, 0.17969742, 0.17943747, 0.17953658, 0.17939524, 0.17944586, 0.17972863, 0.17946644, 0.17937373, 0.17917305, 0.17954633, 0.17963627, 0.17954654, 0.1798709, 0.17940107, 0.17931591, 0.1795245, 0.17906165, 0.17988738, 0.17956138, 0.17947116, 0.17960364, 0.17931122, 0.17988098, 0.17974983, 0.17946145, 0.17947039, 0.17947257, 0.17941712, 0.1798345, 0.17954351, 0.17920409, 0.17956093, 0.17970116, 0.17973208, 0.17950416, 0.17943147, 0.1795452, 0.17940253, 0.17935236, 0.17969121, 0.17979938, 0.17958754, 0.17983536, 0.17985305, 0.179541, 0.1797229, 0.17970067, 0.17973195, 0.1793778, 0.17943478, 0.17987067, 0.17948759, 0.17953801, 0.17957132, 0.17966023, 0.17952012, 0.17929853, 0.17931467, 0.1797087, 0.17912462, 0.17954199, 0.17944223, 0.17976241, 0.17939654, 0.17955254, 0.17955913, 0.1

In [105]:
pred_df_alt_array = pred_df_alt.to_numpy()
pred_df_alt_array[pred_df_alt_array > 0.5] = 1
pred_df_alt_array[pred_df_alt_array <= 0.5] = 0
pred_df_alt_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [102]:
# Get classes from prefetched TF dataset

# https://stackoverflow.com/questions/64622210/how-to-extract-classes-from-prefetched-dataset-in-tensorflow-for-confusion-matri

y_val_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in val_ds_batches: 
   # append true labels
   y_val_true.append(label_batch)
   
print(y_val_true)
# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_val_true], axis = 0)

[<tf.Tensor: shape=(32, 9), dtype=int64, numpy=
array([[0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 0,

In [103]:
correct_labels

<tf.Tensor: shape=(220, 9), dtype=int64, numpy=
array([[0, 0, 1, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 1, 1],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])>

In [106]:
# Print the classification report comparing the true labels and the predicted ones
print(classification_report(correct_labels, pred_df_alt_array))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        32
           1       0.00      0.00      0.00        31
           2       0.00      0.00      0.00        79
           3       0.00      0.00      0.00        24
           4       0.00      0.00      0.00        71
           5       0.00      0.00      0.00        23
           6       0.00      0.00      0.00        40
           7       0.00      0.00      0.00        72
           8       0.00      0.00      0.00        35

   micro avg       0.00      0.00      0.00       407
   macro avg       0.00      0.00      0.00       407
weighted avg       0.00      0.00      0.00       407
 samples avg       0.00      0.00      0.00       407



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Confusion matrix

# print('Confusion Matrix')
# print(confusion_matrix(correct_labels, predicted_labels))

In [ ]:
# Add functions for other performance metrics here


# Layer Visualizations

In [ ]:
# Add code for layer activation visualizations here
